In [ ]:
import math
from typing import Callable, List, Tuple

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp

# interactive plots not working...
# %matplotlib notebook
# %matplotlib widget
%matplotlib inline

In [ ]:
def lorenz_system(
    t: float,
    state: List[float],
    sigma: float,
    rho: float,
    beta: float
) -> List[float]:
    """
    x′=σ(y−x)
    y′=x(ρ−z)−y
    z′=xy−βz
    """
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

def rossler_system(
    t: float,
    state: List[float],
    a: float,
    b: float,
    c: float
) -> List[float]:
    x, y, z = state
    dxdt = -y - z
    dydt = x + a * y
    dzdt = b + z * (x - c)
    return [dxdt, dydt, dzdt]

def thomas_system(
    t: float,
    state: List[float],
    b: float,
) -> List[float]:
    x, y, z = state
    dxdt = np.sin(y) - b * x
    dydt = np.sin(z) - b * y
    dzdt = np.sin(x) - b * z
    return [dxdt, dydt, dzdt]

def run_system(
    f: Callable,
    t_interval: Tuple[float, float],
    init_state: List[float],
    steps: int,
    params,
): 
    t_eval = np.linspace(t_interval[0], t_interval[1], steps)
    x = solve_ivp(f, t_interval, init_state, t_eval=t_eval, args=params.values())
    if x.success != True:
        raise Exception("Solver failed")
    return {'t': x.t, 'state': x.y}
    
def plot_system(
    fig_title: str, 
    f: Callable,
    t_interval: Tuple[float, float],
    init_state: List[float],
    steps: int,
    params,
):
    solution = run_system(f, t_interval, init_state, steps, params)
    x, y, z = solution['state']

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(121, projection='3d')
    ax_table = fig.add_subplot(122)
    ax_table.axis('tight')
    ax_table.axis('off')

    # 3D ax
    ax.plot(x, y, z, linestyle='-', linewidth=0.8, markersize=1)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title(fig_title)

    # Table
    param_names = ['Initial State'] + list(params.keys())
    param_values = [f'[{init_state[0]}, {init_state[1]}, {init_state[2]}]'] + list(params.values())
    table_data = [[name, value] for name, value in zip(param_names, param_values)]

    table = ax_table.table(cellText=table_data, colLabels=['Parameter', 'Value'], loc='center', cellLoc='left')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)

    plt.show()
    return fig
    

In [ ]:
"""
Lorenz.
"""
t_interval = (0, 40)
steps = t_interval[1] * 100

x = 1
y = 1
z = 1
state = [x, y, z]

sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0
params = {
    'sigma': sigma,
    'rho': rho,
    'beta': beta
}

fig_title = 'Lorenz System'
fig = plot_system(fig_title, lorenz_system, (0,100), [1,1,1], steps, params)

In [ ]:
"""Rossler."""
t_interval = (0, 500)
steps = t_interval[1] * 10000

a = 0.2
b = 0.2
c = 5.7

x = 1
y = 1
z = 1
state = [x, y, z]

fig_title = 'Rossler System'
params = {
    'a': a,
    'b': b,
    'c': c
}
fig = plot_system(fig_title, rossler_system, t_interval, state, steps, params)

In [ ]:
"""Thomas."""
t_interval = (0, 500)
steps = t_interval[1] * 1000

b = .2

x = 1
y = 0
z = 0
state = [x, y, z]

fig_title = 'Thomas System'
params = {'b': b}
fig = plot_system(fig_title, thomas_system, t_interval, state, steps, params)